In [1]:
using Test

In [2]:
f = readlines("day7.input");

In [3]:
function parse_line(line)
    if startswith(line, "\$ cd")
        return ("cd", split(line, ' ')[3])
    end
    if startswith(line, "\$ ls")
        return ("ls", nothing)
    end
    if startswith(line, "dir")
        return ("dir", split(line, ' ')[2])
    end
    
    return ("file", parse(Int64, split(line, ' ')[1]))
end

parse_line (generic function with 1 method)

In [4]:
mutable struct Directory
    children::Vector{Directory}
    size::Int64
    parent::Union{Directory, Nothing}
    name::String
end

In [5]:
function get_size(dir::Directory)
    child_total = 0
    if length(dir.children) == 0
        return dir.size
    end
    for child in dir.children
        child_total += get_size(child)
    end
    return child_total + dir.size
end

get_size (generic function with 1 method)

In [6]:
function Directory(name, parent)
    Directory(Vector{Directory}(), 0, parent, name)
end

Directory

In [7]:
function has_subdir(dir1::Directory, dir_name::AbstractString)
    for dir in dir1.children
        if dir_name == dir.name
            return true
        end
    end

    return false
end

has_subdir (generic function with 1 method)

In [15]:
function change_directory(operand::AbstractString, root::Directory, current_dir::Ref{Directory})
    if operand == ".."
        current_dir[] = current_dir[].parent
    elseif operand == "/"
        current_dir[] = root
    else
        if (has_subdir(current_dir[], operand))
            for child in current_dir[].children
                if child.name == operand
                    current_dir[] = child
                    break
                end
            end
        else
            new_dir = Directory(operand, current_dir[])
            push!(current_dir[].children, new_dir)
            current_dir[] = new_dir
        end
    end
end

change_directory (generic function with 1 method)

In [9]:
function process_file(current_dir, file_size)
    current_dir.size += file_size
end

process_file (generic function with 1 method)

In [10]:
function get_score(dir, max_size)
    score = 0
    if get_size(dir) < max_size
        score += dir.size
    end
    for child in dir.children
        score += get_score(child, max_size)
    end

    score
end

get_score (generic function with 1 method)

In [13]:
function solve(input, max_size)
    root = Directory("/", nothing)
    current_dir = root

    for line in input
        operator, operand = parse_line(line)
        if operator == "cd"
            change_directory(operand, root, Ref(current_dir))
        elseif operator == "file"
           process_file(current_dir, operand)
        end
    end

    root
end


solve(f, 10000)

Directory(Directory[Directory(Directory[], 0, Directory(#= circular reference @-3 =#), "bfqzjjct"), Directory(Directory[], 0, Directory(#= circular reference @-3 =#), "cgcqpjpn"), Directory(Directory[], 0, Directory(#= circular reference @-3 =#), "phslrcw"), Directory(Directory[], 0, Directory(#= circular reference @-3 =#), "jnzjq"), Directory(Directory[], 0, Directory(#= circular reference @-3 =#), "lfd"), Directory(Directory[], 0, Directory(#= circular reference @-3 =#), "mlt"), Directory(Directory[], 0, Directory(#= circular reference @-3 =#), "qsdlvr"), Directory(Directory[], 0, Directory(#= circular reference @-3 =#), "nzjfwdt"), Directory(Directory[], 0, Directory(#= circular reference @-3 =#), "sgcg"), Directory(Directory[], 0, Directory(#= circular reference @-3 =#), "bjwzfr")  …  Directory(Directory[], 0, Directory(#= circular reference @-3 =#), "crpfs"), Directory(Directory[], 0, Directory(#= circular reference @-3 =#), "gttptlwj"), Directory(Directory[], 0, Directory(#= circ

In [16]:
@test solve(String.(split("\$ cd /
\$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
\$ cd a
\$ ls
dir e
29116 f
2557 g
62596 h.lst
\$ cd e
\$ ls
584 i
\$ cd ..
\$ cd ..
\$ cd d
\$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k", "\n")), 100000) == 95437


Error During Test at c:\code\advent-of-code-2022\day7.ipynb:1
  Test threw exception
  Expression: solve(String.(split("\$ cd /\n\$ ls\ndir a\n14848514 b.txt\n8504156 c.dat\ndir d\n\$ cd a\n\$ ls\ndir e\n29116 f\n2557 g\n62596 h.lst\n\$ cd e\n\$ ls\n584 i\n\$ cd ..\n\$ cd ..\n\$ cd d\n\$ ls\n4060174 j\n8033020 d.log\n5626152 d.ext\n7214296 k", "\n")), 100000) == 95437
  MethodError: Cannot `convert` an object of type Nothing to an object of type Directory
  Closest candidates are:
    convert(::Type{T}, ::T) where T at Base.jl:61
    Directory(::Any, ::Any) at c:\code\advent-of-code-2022\day7.ipynb:1
    Directory(::Any, ::Any, ::Any, ::Any) at c:\code\advent-of-code-2022\day7.ipynb:2
  Stacktrace:
    [1] setproperty!(x::Base.RefValue{Directory}, f::Symbol, v::Nothing)
      @ Base .\Base.jl:39
    [2] setindex!(b::Base.RefValue{Directory}, x::Nothing)
      @ Base .\refvalue.jl:57
    [3] change_directory(operand::SubString{String}, root::Directory, current_dir::Base.RefValue{Directo

Test.FallbackTestSetException: Test.FallbackTestSetException("There was an error during testing")